In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Geometry import Point3D
import math

def calculate_distance(atom1, atom2):
    """Calculate the distance between two RDKit atoms."""
    pos1 = atom1.GetOwningMol().GetConformer().GetAtomPosition(atom1.GetIdx())
    pos2 = atom2.GetOwningMol().GetConformer().GetAtomPosition(atom2.GetIdx())
    return pos1.Distance(Point3D(pos2.x, pos2.y, pos2.z))

def calculate_angle(atom1, atom2, atom3):
    """Calculate the angle between three RDKit atoms."""
    pos1 = atom1.GetOwningMol().GetConformer().GetAtomPosition(atom1.GetIdx())
    pos2 = atom2.GetOwningMol().GetConformer().GetAtomPosition(atom2.GetIdx())
    pos3 = atom3.GetOwningMol().GetConformer().GetAtomPosition(atom3.GetIdx())
    
    vec1 = pos1 - pos2
    vec2 = pos3 - pos2
    
    angle = math.acos(vec1.DotProduct(vec2) / (vec1.Length() * vec2.Length()))
    return math.degrees(angle)

def find_hbonding_heteroatoms(mol):
    """
    Identify heteroatoms in the molecule that are engaging in hydrogen bonding.
    
    Parameters:
    mol (Chem.Mol): The RDKit molecule.
    
    Returns:
    List of tuples containing donor and acceptor atoms that are engaged in hydrogen bonding.
    """
    hbond_pairs = []
    # Identify potential hydrogen bond donors and acceptors
    donors = [atom for atom in mol.GetAtoms() if atom.GetAtomicNum() in [7, 8] and atom.GetTotalNumHs() > 0]
    acceptors = [atom for atom in mol.GetAtoms() if atom.GetAtomicNum() in [7, 8] and atom.GetTotalNumHs() == 0]

    for donor in donors:
        hydrogen = [nbr for nbr in donor.GetNeighbors() if nbr.GetAtomicNum() == 1][0]
        for acceptor in acceptors:
            if acceptor.GetIdx() != donor.GetIdx():
                distance = calculate_distance(hydrogen, acceptor)
                if distance <= 3.5:  # Common distance threshold for hydrogen bonding
                    angle = calculate_angle(donor, hydrogen, acceptor)
                    if 120 <= angle <= 180:  # Common angle range for hydrogen bonding
                        hbond_pairs.append((donor, acceptor))
    
    return hbond_pairs